Prerequis :
===========
Téléchargement
----------------
wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/release/20190312_biallelic_SNV_and_INDEL/ALL.wgs.shapeit2_integrated_snvindels_v2a.GRCh38.27022019.sites.vcf.gz

Decompression
-------------
gzip -d ALL.wgs.shapeit2_integrated_snvindels_v2a.GRCh38.27022019.sites.vcf.gz

Créer un dossier db à la racine du projet au même niveau que parquet_project

In [30]:
import polars as pl
import duckdb

- Lisez le fichier VCF comme un CSV avec pl.scan_csv
- Sélectionner les colonnes CHROM, POS, REF, ALT
- Écrire le fichier variants.parquet avec sink_parquet

In [31]:
pl.scan_csv(
    "ALL.wgs.shapeit2_integrated_snvindels_v2a.GRCh38.27022019.sites.vcf",
    skip_rows=40,                       # Je saute les 40 premieres lignes de commentaires
    separator="\t",                           # Séparateur TSV
    schema_overrides={"#CHROM": pl.Utf8},         # Je précise le type, sinon la colonne est considérée comme un int
).select([                              # Je sélectionne les colonnes souhaitées
    pl.col("#CHROM").alias("CHROM"),    # Je renomme ici la colonne avec alias
    pl.col("POS"),
    pl.col("REF"),
    pl.col("ALT")]
).sink_parquet(                         # Écriture du fichier parquet 
    "db/variants.parquet"
)

/tmp/ipykernel_214050/2213946102.py:11: DeprecationWarning: The old streaming engine is being deprecated and will soon be replaced by the new streaming engine. Starting Polars version 1.23.0 and until the new streaming engine is released, the old streaming engine may become less usable. For people who rely on the old streaming engine, it is suggested to pin your version to before 1.23.0.

More information on the new streaming engine: https://github.com/pola-rs/polars/issues/20947
  ).sink_parquet(                         # Écriture du fichier parquet


✅ Ce qu’il faut savoir sur le streaming avec Polars
----------------------------------------------------
Le moteur de streaming actuel (avant 1.23) fonctionne, mais il est marqué comme "deprecated" car l’équipe Polars travaille sur une nouvelle version plus robuste.  
Pour le moment, il n’y a pas d’alternative dans Polars pour gérer ce type de fichiers en streaming. Donc même si tu vois le warning, tu peux continuer à l’utiliser.

- Les fonctions scan_csv et sink_csv permettent de faire la transformation du VCF sans le charger en mémoire. 
- Regardez aussi les tailles du fichier. 225Mo pour le fichier parquet et 1.3Go pour son équivalent en CSV. 
- En effet, les fichiers parquets sont compressés naturellement du fait du modèle orienté colonne.

Requête SQL avec DuckDB
-----------------------
À présent essayez de requêter sur ce fichier. Nous pourrions le faire avec pola.rs, mais nous allons plutôt faire une requête SQL en utilisant duckDB.

Pour exécuter une requête SQL sur un fichier parquet, il suffit de considérer le fichier comme le nom d'une table SQL :

In [32]:
duckdb.sql("SELECT count(*) FROM 'db/variants.parquet'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     78229218 │
└──────────────┘

In [33]:
duckdb.sql("SELECT * FROM 'db/variants.parquet' ORDER BY CHROM, POS")

┌─────────┬─────────┬─────────┬─────────┐
│  CHROM  │   POS   │   REF   │   ALT   │
│ varchar │  int64  │ varchar │ varchar │
├─────────┼─────────┼─────────┼─────────┤
│ 1       │   10416 │ CCCTAA  │ C       │
│ 1       │   16103 │ T       │ G       │
│ 1       │   17496 │ AC      │ A       │
│ 1       │   51479 │ T       │ A       │
│ 1       │   51898 │ C       │ A       │
│ 1       │   51928 │ G       │ A       │
│ 1       │   51954 │ G       │ C       │
│ 1       │   54490 │ G       │ A       │
│ 1       │   54669 │ C       │ T       │
│ 1       │   54708 │ G       │ C       │
│ ·       │     ·   │ ·       │ ·       │
│ ·       │     ·   │ ·       │ ·       │
│ ·       │     ·   │ ·       │ ·       │
│ 1       │ 1085650 │ G       │ A       │
│ 1       │ 1085653 │ G       │ A       │
│ 1       │ 1085692 │ G       │ A       │
│ 1       │ 1085713 │ G       │ A       │
│ 1       │ 1085714 │ G       │ A       │
│ 1       │ 1085727 │ G       │ A       │
│ 1       │ 1085807 │ G       │ A 

In [34]:
duckdb.sql("SELECT * FROM 'db/variants.parquet' WHERE REF='A' ORDER BY CHROM, POS")

┌─────────┬─────────┬─────────┬─────────┐
│  CHROM  │   POS   │   REF   │   ALT   │
│ varchar │  int64  │ varchar │ varchar │
├─────────┼─────────┼─────────┼─────────┤
│ 1       │   55365 │ A       │ G       │
│ 1       │   55385 │ A       │ G       │
│ 1       │   57053 │ A       │ T       │
│ 1       │   61987 │ A       │ G       │
│ 1       │   62017 │ A       │ G       │
│ 1       │   66219 │ A       │ T       │
│ 1       │   66259 │ A       │ T       │
│ 1       │   66264 │ A       │ T       │
│ 1       │   66269 │ A       │ T       │
│ 1       │   66275 │ A       │ T       │
│ ·       │     ·   │ ·       │ ·       │
│ ·       │     ·   │ ·       │ ·       │
│ ·       │     ·   │ ·       │ ·       │
│ 1       │ 3351159 │ A       │ G       │
│ 1       │ 3351868 │ A       │ C       │
│ 1       │ 3351893 │ A       │ G       │
│ 1       │ 3352321 │ A       │ G       │
│ 1       │ 3352797 │ A       │ G       │
│ 1       │ 3353146 │ A       │ C       │
│ 1       │ 3353264 │ A       │ G 

À présent, essayons de faire plus compliqué en comptant le nombre de transitions et de transversions. C'est à dire, le nombre de combinaisons A>T, C>G etc ...

In [35]:
q = """
SELECT list_sort([ref,alt]) AS mut, COUNT(*) as count FROM 'db/variants.parquet' 
WHERE len(ref) = 1 AND len(alt)=1 GROUP BY mut
"""

duckdb.sql(q)

┌───────────┬──────────┐
│    mut    │  count   │
│ varchar[] │  int64   │
├───────────┼──────────┤
│ [A, G]    │ 24828822 │
│ [C, G]    │  6315729 │
│ [C, T]    │ 24782079 │
│ [A, T]    │  5140035 │
│ [A, C]    │  6086989 │
│ [G, T]    │  6103978 │
└───────────┴──────────┘

Autres astuces
==============
Le partitionnement
------------------
Niveau performance, c'est déjà bluffant. Mais il existe différentes méthodes d'optimisation pour être plus performant suivant l'usage des données. Le partitionnement consiste à découper votre fichier parquet en plusieurs fichiers parquet depuis une ou plusieurs colonnes. Par exemple, je peux partitionner le fichier parquet variants.parquet par chromosomes. Si je dois chercher un variant sur le chromosome 8, je peux regarder uniquement dans le fichier correspondant. Inutile de parcourir les variants du chromosomes 2.

Construisons une partition sur la colonne chromosome avec duckDB :

In [36]:
duckdb.sql(
    "COPY (SELECT * FROM 'db/variants.parquet') TO 'db/chromosomes' (FORMAT PARQUET, PARTITION_BY (CHROM))"
)

Après avoir exécuté cette requête, vous devriez avoir un dossier chromosomes contenant de nombreux fichiers triés par chromosomes.
Pour sélectionner vos variants depuis ce dossier, il suffit d'utiliser le caractère étoile ou des expressions régulières pour sélectionner les sources de données souhaitées.

Dans l'exemple suivant, je sélectionne tous les variants à partir de tous les fichiers :

In [37]:
duckdb.sql("SELECT * FROM 'db/chromosomes/*/*.parquet'")

┌──────────┬─────────┬─────────┬─────────┐
│   POS    │   REF   │   ALT   │  CHROM  │
│  int64   │ varchar │ varchar │ varchar │
├──────────┼─────────┼─────────┼─────────┤
│ 14795073 │ AC      │ A       │ 1       │
│ 14795117 │ T       │ C       │ 1       │
│ 14795122 │ G       │ T       │ 1       │
│ 14795154 │ T       │ C       │ 1       │
│ 14795187 │ G       │ A       │ 1       │
│ 14795206 │ C       │ T       │ 1       │
│ 14795211 │ T       │ C       │ 1       │
│ 14795263 │ T       │ A       │ 1       │
│ 14795264 │ G       │ A       │ 1       │
│ 14795266 │ G       │ C       │ 1       │
│     ·    │ ·       │ ·       │ ·       │
│     ·    │ ·       │ ·       │ ·       │
│     ·    │ ·       │ ·       │ ·       │
│ 15091644 │ G       │ A       │ 1       │
│ 15091748 │ A       │ G       │ 1       │
│ 15091751 │ C       │ G       │ 1       │
│ 15091891 │ C       │ T       │ 1       │
│ 15091935 │ A       │ G       │ 1       │
│ 15091960 │ G       │ C       │ 1       │
│ 15091980 